# Active Inference for Markov Decision Processes

> This notebook provides an example of the `inferactively` toolbox 

## Environments

The `inferactively` includes environments that follow the openAI `gym` API. Here, we will use a grid-world environment. We will assume that the agent is observing and acting in multiple grid environments simultaneously. This will enable us to demonstrate how *factors* are implemented in `inferactively`.

We assume $N$ grid worlds, each of some arbitary shape $w \ x \ h$. At each time step $t$, the environment generates observations about the agents positions in each grid world. Formally, it generates a vector $[o_0 , ... , o_N]$. 

Agents can take one of 5 actions in each grid world - `{UP, RIGHT, DOWN, LEFT, STAY}`, but here we will sample random actions.

In [20]:
from inferactively.envs import NDGridWorldEnv

env = NDGridWorldEnv(shape=[6, 6], n_dims=2)

obs = env.reset()
for _ in range(5):
    controls = env.sample_action()
    obs = env.step(controls)
    print(f"obs {obs} controls {controls}")

obs [ 9 23] controls [1 1]
obs [10 17] controls [1 0]
obs [16 11] controls [2 0]
obs [17  5] controls [1 0]
obs [17  5] controls [1 0]


### Generative model

Now we have an environment, the next step is to construct an agent's generative model (we will cover learning a model later). `inferactively` contains several useful classes for constructing models.

We consider the following generative model:
$$ p(\mathbf{o}_{1:T}, \mathbf{s}_{1:T}) = \prod_{t=1}^T p(\mathbf{o}_t|\mathbf{s}_t) p(\mathbf{s}_t|\mathbf{s}_{t-1}, \mathbf{u}_{t-1}) $$

Here, $o$ are observations, $s$ are "hidden states" - beliefs about the causes of sensory data - and $u$ are control states.

### Likelihood distribution (`A`)
            
We will begin by considering the likelihood distribution $p(\mathbf{o}_t|\mathbf{s}_t)$, which will be denoted `A` in the code.

To make inference tractble, we factorise the beliefs about hidden states, with one factor for each grid world $N$. In other words, the agent believes that its position in grid $a$ is independent of its position in grid $b$ (which it is).

This likelihood distribution is over $M$ modalities (2). Moreover, there are $N$ hidden state factors (10). In practice, a separate likelihood distribution is maintained for each modality $m$, i.e $p(o_t^m|s_t)$, each of which has the dimensions `(o_m, s_0, ..., s_N)`

For simplicity, we will assume that each hidden state factor maps to a corresponding observation modality, and that this map is an identity mapping. In short, agents have perfect knowledge about where there are in the world. Later, we will introduce uncertainty into the inference procedure.

> _Note: number of hidden states will be `w x h` for each factor_

This can be implemented as follows:

In [44]:
import numpy as np
from inferactively.distributions import Categorical

n_observations = env.n_observations
n_states = env.n_states
print(f"Number of states per factor {n_states} Number of observations per modality {n_observations}")

n_modalities = len(n_observations)
n_factors = len(n_states)
print(f"Number of state factors {n_factors} Number of modalities {n_modalities}")

A = np.empty(n_modalities, dtype="object")
for m in range(n_modalities):
    dist = np.zeros((n_observations[m], *n_states))
    dist[:, :, :] = np.eye(n_states[0])
    A[m] = dist
A = Categorical(values=A)

print(A[0][0, :, :], A[0].shape)

Number of states per factor [36, 36] Number of observations per modality [36, 36]
Number of state factors 2 Number of modalities 2
<Categorical Distribution> 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]] (36, 36, 36)


In [45]:
from inferactively.core import cre

obs = (12, 5)
qs = update_posterior_states(A, obs, return_numpy=False)
print(qs[1].values.transpose())

[[0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778
  0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778
  0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778
  0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778
  0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778
  0.02777778 0.02777778 0.02777778 0.02777778 0.02777778 0.02777778]]
